In [14]:
from crewai import Agent,LLM
from tools.calculator_tools import CalculatorTools
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import SerperDevTool

# Initialize the tool for internet searching capabilities
search_tool = SerperDevTool()
web_scrap_tool = ScrapeWebsiteTool()

llm = LLM(
    model="groq/gemma2-9b-it",
    temperature=0.7
)

class TripAgents():

  def city_selection_agent(self):
    return Agent(
        role='City Selection Expert',
        goal='Select the best city based on weather, season, and prices',
        backstory=
        'An expert in analyzing travel data to pick ideal destinations',
        llm = llm,
        tools=[
            search_tool,
            web_scrap_tool,
        ],
        verbose=True)

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide the BEST insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        llm = llm,
        tools=[
            search_tool,
            web_scrap_tool,
        ],
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and 
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with 
        decades of experience""",
        llm = llm,
        tools=[
            search_tool,
            web_scrap_tool,
            CalculatorTools.calculate,
        ],
        verbose=True)


In [15]:
from crewai import Task
from textwrap import dedent
from datetime import date

class TripTasks:

    def identify_task(self, agent, origin, cities, interests, range):
        return Task(
            description=dedent(f"""
                Analyze and select the best city for the trip based 
                on specific criteria such as weather patterns, seasonal
                events, and travel costs. This task involves comparing
                multiple cities, considering factors like current weather
                conditions, upcoming cultural or seasonal events, and
                overall travel expenses. 
                
                Your final answer must be a detailed
                report on the chosen city, and everything you found out
                about it, including the actual flight costs, weather 
                forecast and attractions.
                {self.__tip_section()}

                Traveling from: {origin}
                City Options: {cities}
                Trip Date: {range}
                Traveler Interests: {interests}
            """),
            agent=agent,
            expected_output="Detailed report on the chosen city including flight costs, weather forecast, and attractions"
        )

    def gather_task(self, agent, origin, interests, range):
        return Task(
            description=dedent(f"""
                As a local expert on this city you must compile an 
                in-depth guide for someone traveling there and wanting 
                to have THE BEST trip ever!
                Gather information about key attractions, local customs,
                special events, and daily activity recommendations.
                Find the best spots to go to, the kind of place only a
                local would know.
                This guide should provide a thorough overview of what 
                the city has to offer, including hidden gems, cultural
                hotspots, must-visit landmarks, weather forecasts, and
                high level costs.
                
                The final answer must be a comprehensive city guide, 
                rich in cultural insights and practical tips, 
                tailored to enhance the travel experience.
                {self.__tip_section()}

                Trip Date: {range}
                Traveling from: {origin}
                Traveler Interests: {interests}
            """),
            agent=agent,
            expected_output="Comprehensive city guide including hidden gems, cultural hotspots, and practical travel tips"
        )

    def plan_task(self, agent, origin, interests, range):
        return Task(
            description=dedent(f"""
                Expand this guide into a full 7-day travel 
                itinerary with detailed per-day plans, including 
                weather forecasts, places to eat, packing suggestions, 
                and a budget breakdown.
                
                You MUST suggest actual places to visit, actual hotels 
                to stay and actual restaurants to go to.
                
                This itinerary should cover all aspects of the trip, 
                from arrival to departure, integrating the city guide
                information with practical travel logistics.
                
                Your final answer MUST be a complete expanded travel plan,
                formatted as markdown, encompassing a daily schedule,
                anticipated weather conditions, recommended clothing and
                items to pack, and a detailed budget, ensuring THE BEST
                TRIP EVER. Be specific and give it a reason why you picked
                each place, what makes them special! {self.__tip_section()}

                Trip Date: {range}
                Traveling from: {origin}
                Traveler Interests: {interests}
            """),
            agent=agent,
            expected_output="Complete expanded travel plan with daily schedule, weather conditions, packing suggestions, and budget breakdown"
        )

    def __tip_section(self):
        return "If you do your BEST WORK, I'll tip you $100!"


In [17]:
from crewai import Crew
from textwrap import dedent
from trip_agents import TripAgents
from trip_tasks import TripTasks

# from dotenv import load_dotenv
# load_dotenv()

class TripCrew:

  def __init__(self, origin, cities, date_range, interests):
    self.cities = cities
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    city_selector_agent = agents.city_selection_agent()
    local_expert_agent = agents.local_expert()
    travel_concierge_agent = agents.travel_concierge()

    identify_task = tasks.identify_task(
      city_selector_agent,
      self.origin,
      self.cities,
      self.interests,
      self.date_range
    )
    gather_task = tasks.gather_task(
      local_expert_agent,
      self.origin,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      travel_concierge_agent, 
      self.origin,
      self.interests,
      self.date_range
    )

    crew = Crew(
      agents=[
        city_selector_agent, local_expert_agent, travel_concierge_agent
      ],
      tasks=[identify_task, gather_task, plan_task],
      verbose=True
    )

    result = crew.kickoff()
    return result

if __name__ == "__main__":
  print("## Welcome to Trip Planner Crew")
  print('-------------------------------')
  location = "pune"
  cities = "Dubai"
  date_range = "10th jan 2025 to 25th jan 2025"
  interests = "Technology, Temple"
  
  trip_crew = TripCrew(location, cities, date_range, interests)
  result = trip_crew.run()
  print("\n\n########################")
  print("## Here is you Trip Plan")
  print("########################\n")
  print(result)


2025-01-02 20:06:51,619 - 7180 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Welcome to Trip Planner Crew
-------------------------------
# Agent: City Selection Expert
## Task: 
Analyze and select the best city for the trip based 
on specific criteria such as weather patterns, seasonal
events, and travel costs. This task involves comparing
multiple cities, considering factors like current weather
conditions, upcoming cultural or seasonal events, and
overall travel expenses. 

Your final answer must be a detailed
report on the chosen city, and everything you found out
about it, including the actual flight costs, weather 
forecast and attractions.
If you do your BEST WORK, I'll tip you $100!

Traveling from: pune
City Options: Dubai
Trip Date: 10th jan 2025 to 25th jan 2025
Traveler Interests: Technology, Temple



2025-01-02 20:06:52,405 - 7180 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************daGQ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: City Selection Expert
## Task: 
Analyze and select the best city for the trip based 
on specific criteria such as weather patterns, seasonal
events, and travel costs. This task involves comparing
multiple cities, considering factors like current weather
conditions, upcoming cultural or seasonal events, and
overall travel expenses. 

Your final answer must be a detailed
report on the chosen city, and everything you found out
about it, including the actual flight costs, weather 
forecast and attractions.
If you do your BEST WORK, I'll tip you $100!

Traveling from: pune
City Options: Dubai
Trip Date: 10th jan 2025 to 25th jan 2025
Traveler Interests: Technology, Temple



2025-01-02 20:06:52,924 - 7180 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************daGQ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: City Selection Expert
## Task: 
Analyze and select the best city for the trip based 
on specific criteria such as weather patterns, seasonal
events, and travel costs. This task involves comparing
multiple cities, considering factors like current weather
conditions, upcoming cultural or seasonal events, and
overall travel expenses. 

Your final answer must be a detailed
report on the chosen city, and everything you found out
about it, including the actual flight costs, weather 
forecast and attractions.
If you do your BEST WORK, I'll tip you $100!

Traveling from: pune
City Options: Dubai
Trip Date: 10th jan 2025 to 25th jan 2025
Traveler Interests: Technology, Temple



2025-01-02 20:06:53,292 - 7180 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************daGQ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************daGQ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [23]:
from crewai import Agent,LLM
from tools.calculator_tools import CalculatorTools
from crewai_tools import ScrapeWebsiteTool
from crewai_tools import SerperDevTool

# Initialize the tool for internet searching capabilities
search_tool = SerperDevTool()
web_scrap_tool = ScrapeWebsiteTool()

llm = LLM(
    model="groq/llama3-8b-8192",
    temperature=0.7
)

city_selection_agent = Agent(
        role='City Selection Expert',
        goal='Select the best city based on weather, season, and prices',
        backstory=
        'An expert in analyzing travel data to pick ideal destinations',
        llm = llm,
        tools=[
            search_tool,
            web_scrap_tool,
        ],
        verbose=True)

local_expert_agent = Agent(
        role='Local Expert at this city',
        goal='Provide the BEST insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        llm = llm,
        tools=[
            search_tool,
            web_scrap_tool,
        ],
        verbose=True)

travel_budget_agent = Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and 
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with 
        decades of experience""",
        llm = llm,
        tools=[
            search_tool,
            web_scrap_tool,
            CalculatorTools.calculate,
        ],
        verbose=True)



from crewai import Task
from textwrap import dedent
from datetime import date

city_selection_task = Task(
    description="""
        Analyze and select the best city for the trip based 
        on specific criteria such as weather patterns, seasonal
        events, and travel costs. This task involves comparing
        multiple cities, considering factors like current weather
        conditions, upcoming cultural or seasonal events, and
        overall travel expenses. 
        
        Your final answer must be a detailed
        report on the chosen city, and everything you found out
        about it, including the actual flight costs, weather 
        forecast and attractions.
        If you do your BEST WORK, I'll tip you $100!

        Traveling from: {origin}
        City Options: {cities}
        Trip Date: {range}
        Traveler Interests: {interests}
    """,
    agent=city_selection_agent,
    expected_output="Detailed report on the chosen city including flight costs, weather forecast, and attractions"
)

local_expert_task = Task(
    description="""
        As a local expert on this city you must compile an 
        in-depth guide for someone traveling there and wanting 
        to have THE BEST trip ever!
        Gather information about key attractions, local customs,
        special events, and daily activity recommendations.
        Find the best spots to go to, the kind of place only a
        local would know.
        This guide should provide a thorough overview of what 
        the city has to offer, including hidden gems, cultural
        hotspots, must-visit landmarks, weather forecasts, and
        high level costs.
        
        The final answer must be a comprehensive city guide, 
        rich in cultural insights and practical tips, 
        tailored to enhance the travel experience.
        If you do your BEST WORK, I'll tip you $100!

        Trip Date: {range}
        Traveling from: {origin}
        Traveler Interests: {interests}
    """,
    agent=local_expert_agent,
    expected_output="Comprehensive city guide including hidden gems, cultural hotspots, and practical travel tips"
)

travel_budget_task = Task(
    description="""
        Expand this guide into a full 7-day travel 
        itinerary with detailed per-day plans, including 
        weather forecasts, places to eat, packing suggestions, 
        and a budget breakdown.
        
        You MUST suggest actual places to visit, actual hotels 
        to stay and actual restaurants to go to.
        
        This itinerary should cover all aspects of the trip, 
        from arrival to departure, integrating the city guide
        information with practical travel logistics.
        
        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER. Be specific and give it a reason why you picked
        each place, what makes them special! If you do your BEST WORK, I'll tip you $100!

        Trip Date: {range}
        Traveling from: {origin}
        Traveler Interests: {interests}
    """,
    agent=travel_budget_agent,
    expected_output="Complete expanded travel plan with daily schedule, weather conditions, packing suggestions, and budget breakdown"
)



from crewai import Crew
from textwrap import dedent
from trip_agents import TripAgents
from trip_tasks import TripTasks


crew = Crew(
    agents=[city_selection_agent, local_expert_agent, travel_budget_agent],
    tasks=[city_selection_task, local_expert_task, travel_budget_task],
    verbose=True
)

inp={'origin': "Pune", 
        'cities': "Dubai",
        "range" : "10th jan 2025 to 15th jan 2025",
        "interests" : "Technology, Temple"}

result = crew.kickoff(inputs=inp)




2025-01-02 22:07:19,823 - 7180 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: City Selection Expert
## Task: 
        Analyze and select the best city for the trip based 
        on specific criteria such as weather patterns, seasonal
        events, and travel costs. This task involves comparing
        multiple cities, considering factors like current weather
        conditions, upcoming cultural or seasonal events, and
        overall travel expenses. 
        
        Your final answer must be a detailed
        report on the chosen city, and everything you found out
        about it, including the actual flight costs, weather 
        forecast and attractions.
        If you do your BEST WORK, I'll tip you $100!

        Traveling from: Pune
        City Options: Dubai
        Trip Date: 10th jan 2025 to 15th jan 2025
        Traveler Interests: Technology, Temple
    
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Input should be a v

In [24]:
print(result)

**7-Day Travel Itinerary for Dubai**

**Day 1: Arrival in Dubai (10th January 2025)**

* Morning: Arrive at Dubai International Airport and take a taxi to your hotel, the 3-star Rove Dubai Marina (approx. INR 2,500/USD 35).
* Afternoon: Check-in to your hotel and freshen up. Visit the nearby Dubai Marina promenade and take a stroll around the marina.
* Evening: Head to the Dubai Mall and enjoy the evening atmosphere. Visit the Dubai Fountain Show and take a walk around the mall.

**Day 2: Exploring Dubai (11th January 2025)**

* Morning: Start the day with a visit to the Burj Khalifa, the world's tallest building. Take the high-speed elevator to the 124th floor and enjoy the stunning views of the city.
* Afternoon: Visit the Dubai Museum and learn about the city's history and culture.
* Evening: Head to the Global Village and enjoy the cultural entertainment and food from around the world.

**Day 3: Technology and Innovation (12th January 2025)**

* Morning: Visit the Dubai Internet Ci